# <center> **Home Credit Default Risk Assessment**
# <center> **Bureau | Bureau_Balance Datasets**

# **Introduction**

In this part of the project, I aggregate features in the bureau and bureau-balance tables and merged them into a table I call bureau. Through the aggregation new features are created that I will merge with the main application_train table in a later stage.

# **Libraries**

In [1]:
import pandas as pd
import numpy as np

from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import ArbitraryNumberImputer

import functions
import importlib
importlib.reload(functions)

import warnings

# **Display**

In [2]:
%matplotlib inline

pd.options.display.max_rows = 300000
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 500

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option('display.max_rows', 200)

size = 20

## **Load Data**

In [3]:
bureau = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Risk\Data\bureau.csv",
    index_col=False
)

balance = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Risk\Data\bureau_balance.csv",
    index_col=False
)

## **Reduce Memory Usage**

Changing datatypes to a lower level to save on system resources.

In [4]:
bureau = functions.reduce_memory_usage(bureau)

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%


In [5]:
balance = functions.reduce_memory_usage(balance)

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%


## **Remove Infinity Values**

Replace inf and -inf with NAN.

In [6]:
bureau.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
balance.replace([np.inf, -np.inf], np.nan, inplace=True)

## **Imputation**

Imputation of missing values. Numerical missing values were imputed with an arbitrary number. Categorical missing values were imputed by "UNKNOWN". This is a deliberate choice to not introduce new patterns in the data.

In [8]:
ani = ArbitraryNumberImputer(arbitrary_number=-99999)
ani.fit(bureau)
bureau = ani.transform(bureau)

In [9]:
ani = ArbitraryNumberImputer(arbitrary_number=-99999)
ani.fit(balance)
balance = ani.transform(balance)

In [10]:
ci = CategoricalImputer(imputation_method='missing', fill_value='UNKNOWN')
ci.fit(bureau)
bureau = ci.transform(bureau)

In [11]:
ci = CategoricalImputer(imputation_method='missing', fill_value='UNKNOWN')
ci.fit(balance)
balance = ci.transform(balance)

## **Aggregation**

Aggregation, feature creation in both bureau and bureau_balance tables. I merge bureau and bureau_balance tables after these operations. 

In [12]:
balance = balance.groupby('SK_ID_BUREAU').agg(
    NUM_MONTHS=('MONTHS_BALANCE', 'count'),
    SUM_STATUSES=('STATUS', lambda x: (x.isin(['1', '2', '3', '4', '5'])).sum()),
    MAX_DPD=('STATUS', lambda x: x.replace({'C': -1, 'X': -1}).astype(int).max()),
    NUM_CLOSED=('STATUS', lambda x: (x == 'C').sum()),
    NUM_UNKNOWN=('STATUS', lambda x: (x == 'X').sum()),
)

bureau = bureau.merge(balance, on='SK_ID_BUREAU', how='left')

bureau = bureau.groupby('SK_ID_CURR').agg(
    NUM_LOANS=('SK_ID_BUREAU', 'count'),
    TOTAL_NUM_MONTHS=('NUM_MONTHS', 'sum'),
    TOTAL_SUM_STATUSES=('SUM_STATUSES', 'sum'),
    AVG_MAX_DPD=('MAX_DPD', 'mean'),
    TOTAL_NUM_CLOSED=('NUM_CLOSED', 'sum'),
    TOTAL_NUM_UNKNOWN=('NUM_UNKNOWN', 'sum'),
    NUM_ACTIVE_LOANS=('CREDIT_ACTIVE', lambda x: (x == 'Active').sum()),
    TOTAL_DEBIT=('AMT_CREDIT_SUM_DEBT', 'sum'),
    TOTAL_CREDIT_AMT=('AMT_CREDIT_SUM', 'sum'),
    DEBT_CREDIT_RATIO=('AMT_CREDIT_SUM_DEBT', lambda x: x.sum() / (bureau.loc[x.index, 'AMT_CREDIT_SUM'].sum() + 1e-5)),
    TOTAL_OVERDUE=('AMT_CREDIT_SUM_OVERDUE', 'sum'),
    MAX_OVERDUE=('AMT_CREDIT_MAX_OVERDUE', 'max'),
    AVG_DAYS_OVERDUE=('CREDIT_DAY_OVERDUE', 'mean'),
    NUM_PROLONGED_LOANS=('CNT_CREDIT_PROLONG', lambda x: (x > 0).sum()),
)

In [13]:
bureau = bureau.reset_index()
bureau.head()

,SK_ID_CURR,NUM_LOANS,TOTAL_NUM_MONTHS,TOTAL_SUM_STATUSES,AVG_MAX_DPD,TOTAL_NUM_CLOSED,TOTAL_NUM_UNKNOWN,NUM_ACTIVE_LOANS,TOTAL_DEBIT,TOTAL_CREDIT_AMT,DEBT_CREDIT_RATIO,TOTAL_OVERDUE,MAX_OVERDUE,AVG_DAYS_OVERDUE,NUM_PROLONGED_LOANS
0,100001,7,172.0,1.0,0.142857,110.0,30.0,3,596686.5,1.453365e+06,0.410555,0.0,-99999.00000,0.0,0
1,100002,8,110.0,27.0,0.750000,23.0,15.0,2,-54216.0,8.650556e+05,-0.062673,0.0,5043.64502,0.0,0
2,100003,4,0.0,0.0,NaN,0.0,0.0,1,0.0,1.017400e+06,0.000000,0.0,0.00000,0.0,0
3,100004,2,0.0,0.0,NaN,0.0,0.0,0,0.0,1.890378e+05,0.000000,0.0,0.00000,0.0,0
4,100005,3,21.0,0.0,0.000000,5.0,2.0,2,568408.5,6.571260e+05,0.864992,0.0,0.00000,0.0,0


# **Save Dataframe as CSV File**

A new dataframe is created to be used in later parts of this project.

In [14]:
bureau.to_csv(r"C:\Users\Dell\Documents\AI\Risk\Data\Data\bureau 24.csv", index=False)

# **Summary**

> * **bureau Table** — I used domain knowledge gained from research to aggregate and create new features from those in the bureau table. 
> * **bureau_balance Table** — I used domain knowledge gained from research to aggregate and create new features from those in the bureau-balance table.
> * **Merging the Two Tables** — I merged the two aggregated tables into a single table, I called bureau.
> * **Merge** — I will later merge the new table with the main application_train table in Notebook 10.0.